In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import acquire
import summarize
import prepare

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# ACQUIRE

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.logerror.mean()

In [ ]:
df[(df.bathroomcnt > df.bedroomcnt)].logerror.mean()

In [ ]:
df[(df.bathroomcnt < df.bedroomcnt)].logerror.mean()

In [ ]:
df[(df.bathroomcnt == df.bedroomcnt)].logerror.mean()

#### Keep the Master Latitude Longitude DataFrame

In [ ]:
lat_long = df[['latitude','longitude']]

In [ ]:
len(lat_long)

# Prepare

### Get rid of nulls. Heating will get imputed. Null rows will be dropped.

In [ ]:
#The columns that have to do with the building themselves. No location data except for lat-long.
house_vars = ['bathroomcnt','bedroomcnt','calculatedfinishedsquarefeet','heatingorsystemtypeid','lotsizesquarefeet','yearbuilt','structuretaxvaluedollarcnt', 'taxvaluedollarcnt','latitude','longitude','logerror']

In [ ]:
house_vars

In [ ]:
structures_df = df[house_vars]

In [ ]:
structures_df.heatingorsystemtypeid.fillna(2, inplace=True)

In [ ]:
structures_df.dropna(inplace=True)

In [ ]:
summarize.nulls_by_col(structures_df)

### Split into the train and test sets

In [ ]:
train, test = train_test_split(structures_df, train_size = .8, random_state = 123)

In [ ]:
train.drop(['latitude','longitude'],axis=1,inplace=True)

test.drop(['latitude','longitude'],axis=1,inplace=True)

In [ ]:
train.head()

### USE STANDARD SCALING

In [ ]:
standard_train, standard_test, standard_object = prepare.standardize_train_test(train, test)

### Drop the upper outliers

In [ ]:
no_outliers = prepare.remove_upper_outliers(standard_train.calculatedfinishedsquarefeet, train)
no_outliers = prepare.remove_upper_outliers(standard_train.lotsizesquarefeet, train)

In [ ]:
no_outliers.shape

In [ ]:
kmean = KMeans(n_clusters=3)

In [ ]:
kmean.fit(standard_train)

In [ ]:
predictions3 = kmean.labels_

standard_train['cluster_labels3'] = predictions3

In [ ]:
np.unique(predictions3, return_counts=True)

In [ ]:
standard_train.groupby('cluster_labels3').mean()

In [ ]:
## THE 2nd Cluster does the best. Very low log error.

In [ ]:
standard_train[['latitude','longitude']] = lat_long

In [ ]:
standard_train.head()

In [ ]:
sns.scatterplot(data=standard_train, x='longitude', y='latitude', hue='cluster_labels3')

In [ ]:
sns.scatterplot(data=df, x='longitude', y='latitude')

In [ ]:
import cluster

In [ ]:
cluster.show_clusters_on_map()